In [1]:
#modules
import numpy
import pandas as pd
import json
import urllib2
import datetime
import time

In [2]:
#API data from ticker

def getAPIData(ticker):
    websource = urllib2.urlopen('https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol='+ticker+'&outputsize=full&apikey=CBU3HK62CDA53MJU')
    data1 = json.loads(websource.read().decode())
    data = data1['Time Series (Daily)']
    df = pd.DataFrame.from_dict(data, orient = 'index')
    return df

In [3]:
#CCY to GBP

CCY_list =[]

def CCYtoGBP(df_pos_CCY, df_API_CCY, headings_list_CCY):
    for index, row in df_pos_CCY.iterrows():
        if row['CCY'] <> 'GBP':
            addCCY = True
            for x in CCY_list:
                if row['CCY'] == x:
                    addCCY = False
            if addCCY == True:
                CCY_list.append(row['CCY'])
    
    for i in CCY_list:
        try:
            websource = urllib2.urlopen('https://www.alphavantage.co/query?function=FX_DAILY&from_symbol='+i+'&to_symbol=GBP&outputsize=full&apikey=CBU3HK62CDA53MJU')
            data_CCY = json.loads(websource.read().decode())
            data_CCY = data_CCY['Time Series FX (Daily)']
            df_temp_CCY = pd.DataFrame.from_dict(data_CCY, orient = 'index')
            time.sleep(20)
            df_temp_CCY = df_temp_CCY['4. close']
            df_temp_CCY = df_temp_CCY.astype(float) #numbers to float
            df_temp_CCY.fillna(1, inplace = True)
            df_temp_CCY.index = pd.to_datetime(df_temp_CCY.index) # index str to dates         
            #df_API_CCY = pd.concat([df_API_CCY, df_temp_CCY], axis=1, sort=False)
            #df_API_CCY = pd.concat([df_API_CCY, df_temp_CCY.reindex(df_API_CCY.index)], axis=1, sort=False)
            #df_API_CCY = pd.merge(df_API_CCY, df_temp_CCY, left_index=True, right_index=True
            df_API_CCY = df_API_CCY.join(df_temp_CCY)
            headings_list.append(i)
            df_API_CCY.columns = [headings_list] #add headings
            #df_API_CCY.rename({'4. close': i}, axis=1, inplace=True)
            
            print 'Proceesed CCY '+ i
        except Exception, e:
            print 'Error on CCY ' + i +' message: ' +str(e)
    
    return df_API_CCY   

#CCYtoGBP(df_pos, df_API)


In [4]:
#position data
filename = 'Data_trial.csv'
#filename = 'Data.csv'

df_pos = pd.read_csv(filename, delimiter=';')
df_pos['Initial_Value'] = pd.Series(0.0, index=df_pos.index)
df_pos['Current_Value'] = pd.Series(0.0, index=df_pos.index)
df_pos['Delta'] = pd.Series(0.0, index=df_pos.index)
df_pos['Percent_change'] = pd.Series(0.0, index=df_pos.index)
df_pos['Volatility'] = pd.Series(0.0, index=df_pos.index)
#df_pos['CCY_conversion'] = pd.Series(1.0, index=df_pos.index)

#df_pos = df_pos.astype(float) #numbers to float
#df_pos.Open = pd.to_datetime(df_pos.Open) # str to date

#df_pos.head()

In [5]:
# GBP CCY value, initial value, current value, delta, % change, volatility 
# per stock

def posData(df_pos_pos, df_API_pos, date_index):
    ccy_rate_init = 1.0
    ccy_rate_curr = 1.0
    #date_index = df_API_pos.index.max
    #date_index = '2019-07-30'
    for index, row in df_pos_pos.iterrows():
        if row['CCY'] <> 'GBP':
            ccy_rate_init = df_API_pos.loc[row['Open'],row['CCY']].values[0,0]
            #ccy_rate_curr = df_API_pos.loc[df_API_pos.index[-1],row['CCY']]
            ccy_rate_curr = df_API_pos.loc[date_index,row['CCY']].values[0,0]
            #print df_API_pos.loc[df_API_pos.index.max,row['CCY']]
            #print df_API_pos.index.max
        try:
            ip_df = df_API_pos.loc[row['Open'],row['Ticker']]*row['Qty']*ccy_rate_init
            ip = ip_df.iat[0,0]
            df_pos_pos.iat[index, 8] = ip
            cp_df = df_API_pos.loc[date_index,row['Ticker']]*row['Qty']*ccy_rate_curr
            cp = cp_df.iat[0,0]
            df_pos_pos.iat[index, 9] = cp
            #print cp
            
            if row['Type'] == 'Short':
                delta = ip - cp
                df_pos_pos.iat[index, 10] = delta
            if row['Type'] == 'Long':
                delta = cp - ip
                df_pos_pos.iat[index, 10] = delta 
                 
            pc = (delta / ip)
            df_pos_pos.iat[index, 11] = pc
            
        except:
            print 'Error on '+str(row['Ticker'])
            
    return df_pos_pos

#posData(df_pos, df_API).head()
        

In [6]:
#daily totals per strategy

strategy_list =[]

def strategyData(df_pos_strat, df_API_strat, headings_list_strat):
    for index, row in df_pos_strat.iterrows():
        addStrat = True
        for x in strategy_list:
            if row['Strategy'] == x:
                addStrat = False
        if addStrat == True:
            strategy_list.append(row['Strategy'])
            df_API_strat[row['Strategy']] = pd.Series(0.0, index=df_API_strat.index)
    

    for y in strategy_list: 
        print 'Strategy: ' + y
        for index_API, row_API in df_API_strat.iterrows():
            strategy_total =0.0
            ccy_rate = 1.0
            
            for index, row in df_pos_strat.iterrows():
                row['Open'] = pd.to_datetime(row['Open'])
                if index_API >= row['Open']:
                    if row['Strategy'] == y:    
                        try:        
                            if row['CCY'] <> 'GBP':
                                ccy_rate = df_API_strat.loc[index_API,row['CCY']].values[0]
                            if row['Type'] == 'Long':
                                strategy_total = strategy_total + (ccy_rate * row['Qty'] * df_API_strat.loc[index_API,row['Ticker']].values[0])
                            if row['Type'] == 'Short':
                                open_value = row['Qty'] * df_API_strat.loc[row['Open'],row['Ticker']].values[0]
                                strategy_total = strategy_total + open_value + open_value - (ccy_rate * row['Qty'] * df_API_strat.loc[index_API,row['Ticker']].values[0])
                        except:
                            error_strat = 1
                            #print 'Error on ' + row['Ticker']   
            row_API[y] = strategy_total
            #print strategy_total
            
    return df_API_strat

#a = strategyData(df_pos, df_API,headings_list)              
#a.head()

In [7]:
#stock data
df_API = pd.DataFrame()
headings_list = []
current_date = '2019-09-06'

for index, row in df_pos.iterrows():
    try:
        df_temp = getAPIData(row['Ticker'])
        print 'Processed '+ row['Ticker']
        time.sleep(20)
        df_temp = df_temp['4. close']
        df_API = pd.concat([df_API, df_temp], axis=1, sort=False)
        headings_list.append(row['Ticker'])
    except Exception, e:
        print 'Error on ' + row['Ticker'] +' message: ' +str(e)

df_API.columns = [headings_list] #add headings 
df_API = df_API.astype(float) #numbers to float
df_API.index = pd.to_datetime(df_API.index) # index str to dates
df_API.fillna(0, inplace = True)

df_API = CCYtoGBP(df_pos, df_API, headings_list)
df_pos = posData(df_pos, df_API, current_date)
df_API = strategyData(df_pos, df_API, headings_list)

#df_pos.head()

Processed LON:RIO
Processed LON:AAL
Processed NASDAQ:FB
Processed NASDAQ:AMZN
Proceesed CCY USD
Strategy: GB_Mining
Strategy: FAANGS_Short


In [12]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from datetime import datetime, date, time
import datetime as dt
import time
import matplotlib as mpl

mpl.rcParams.update({'font.size': 8})

filename = 'out_API.csv'

df_pos = pd.read_csv(filename, delimiter=';')

def stratPlot(x,y, labelp):
	plt.plot(x,y, marker=',',linewidth=0.7, label = labelp)

def strategyPlot(st_list, df):
	df = df[df.index.to_pydatetime() > dt.datetime(2017,3,1)]
	df = df[(df < 12000).all(1)] 

	x = np.array(df.index.to_pydatetime(), dtype=np.datetime64)

	for i in st_list:
		y = df[i]#df[[i]].to_numpy()
        print y
		stratPlot(x, y, i)	

	plt.xlabel('Date')
	plt.ylabel('Value(USD)')
	plt.title('Strategy')
	plt.xticks(rotation=20)
	plt.legend()

	#plt.savefig('Strategy ' + str(datetime.today().strftime('%Y-%m-%d')) +'.pdf')
	
	plt.show()
    
strategyPlot(strategy_list, df_API)

IndentationError: unexpected indent (<ipython-input-12-7fef3b4e86d9>, line 23)

In [ ]:
b = pd.DataFrame()

b = df_API[strategy_list]
c = b.loc[b.index >= '2019-07-01']

#d = c[(c != 0).all(1)]
f = c[(c < 12000).all(1)]
#d = c[(c[strategy_list[0]] != 0).any()]
f.plot()

In [ ]:
b = pd.DataFrame()

b = df_API[strategy_list]
c = b.loc[b.index >= '2017-03-01']

#d = c[(c != 0).all(1)]
f = c[(c < 12000).all(1)]
#d = c[(c[strategy_list[0]] != 0).any()]
f.plot()

In [ ]:
df_pos.head(12)

In [ ]:
#table = 
pd.pivot_table(df_pos, values=['Delta', 'Initial_Value', 'Current_Value'], index=['Strategy'], aggfunc=sum)
#print table

In [ ]:

f.to_csv("out.csv")
df_API.to_csv("out_API.csv")


In [ ]:
df_pos.to_csv("out_pos.csv")

In [10]:
print y

NameError: name 'y' is not defined